In [6]:
import pandas as pd
# import os
# import shutil

# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text
# from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

df = pd.read_csv(r'sarcasm_subset.csv')
frames = [df]
full = pd.concat(frames)
full

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,Bushie the other Bushie the next Bushie the af...,selicos,politics,606,-1,-1,2016-10,2016-10-13 02:39:41,"Don't you mean, Bushie the Younger?"
1,1,Congrats on getting such damage before they we...,SenGenketsu,Robocraft,1,-1,-1,2016-10,2016-10-27 16:51:51,"So, Flak will be a thing again? Than i have to..."
2,0,Keeper of the Grove too,pucykoks,hearthstone,14,14,0,2016-09,2016-09-13 21:33:07,Especially when you compare it to dalaran Mage...
3,1,'This would be like my form of alcoholism' cle...,redditzendave,politics,7,-1,-1,2016-10,2016-10-21 19:16:31,See It: Trump hires teen because she's beautif...
4,1,Great input,Karieo,wow,2,2,0,2016-09,2016-09-19 20:54:15,you swap to druid.
...,...,...,...,...,...,...,...,...,...,...
99995,0,"Alright lmao, a bit *too* gay for my tastes",ScootaliciousScooter,teenagers,2,-1,-1,2016-12,2016-12-30 04:05:43,"This is just getting too far, sort of like you..."
99996,1,Rip this is the farthest I've ever gotten,EmceeSexy,teenagers,1,-1,-1,2016-12,2016-12-05 02:18:19,ok
99997,1,Was expecting a tutorial on how to doxx your d...,Orrangejuiced,h1z1,3,-1,-1,2016-12,2016-12-07 04:48:41,Ninja tutorial
99998,1,He just wants more time to prepare for playing...,bamachine,CFB,2,-1,-1,2016-11,2016-11-23 03:55:23,Coaches Poll: Most feel Jimbo Fisher leaves fo...


In [53]:
text_data = list(df["comment"])[:5000]
class_data = df["label"].to_numpy()[:5000]
for i in range(len(class_data)):
    if class_data[i]=='notsarc':
        class_data[i]=0
    else:
        class_data[i]=1
        
# for i in text_data:
#     print(type(i))

In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained BERT model and tokenizer
model = torch.load('BERT.pth')
model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the classification model
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        logits = self.fc(pooled_output)
        return logits

# Load the training data
# train_texts = ["Example sentence 1", "Example sentence 2", ...]
# train_labels = [0, 1, ...]

# Tokenize the text data
train_encodings = tokenizer(list(text_data), padding=True, truncation=True, max_length=128)
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(list(class_data))
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

acc = []
for idx, batch in enumerate(train_loader):
    outputs = model(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
    outputs = torch.argmax(outputs, axis = 1)
    acc.append((outputs.cpu() == batch[2]).float().sum().numpy())
    
print("Accuracy: ", (sum(acc) / 5000) * 100, "%")

Accuracy:  77.34 %


In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained BERT model and tokenizer
model = torch.load('RoBERTa.pth')
model.to(device)
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Define the classification model
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        logits = self.fc(pooled_output)
        return logits

# Load the training data
# train_texts = ["Example sentence 1", "Example sentence 2", ...]
# train_labels = [0, 1, ...]

# Tokenize the text data
train_encodings = tokenizer(list(text_data), padding=True, truncation=True, max_length=128)
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(list(class_data))
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

acc = []
for idx, batch in enumerate(train_loader):
    outputs = model(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
    outputs = torch.argmax(outputs, axis = 1)
    acc.append((outputs.cpu() == batch[2]).float().sum().numpy())
    
print("Accuracy: ", (sum(acc) / 5000) * 100, "%")

Accuracy:  64.86 %
